# This notebook will generate the output of the apoptosis model

---
## Apoptosis Model
### Model scope
This model was published by Albeck *et al.* 2008 (https://doi.org/10.1371/journal.pbio.0060299) and modified by Loriaux *et al.* 2013 to include expression and degradation of most chemical species (https://doi.org/10.1371/journal.pcbi.1002901). The key finding of this model is the variable delay switch-like behavior. Scope is shown below:

In [ ]:
using DifferentialEquations, DataFrames
using Plots, StatsPlots
using CSV, Tables
using Sundials

In [ ]:
#set up where CSV2Julia is
locationOfCSV2Julia="../CSV2JuliaDiffEq-master/csv2model.py"

# Averaged cell line

In [ ]:
#identify the three CSV sheets that describe the model
reactionsFile="moduleDefinitionFiles/averaged/reactions.csv"
parametersFile="moduleDefinitionFiles/averaged/parameters.csv"
rateLawsFile="moduleDefinitionFiles/averaged/rateLaws.csv"

#build a command to execute csv2julia
location=[locationOfCSV2Julia]
arguments=[reactionsFile, parametersFile, rateLawsFile,"odeApoptosis.jl"]
cmd=`python3 $locationOfCSV2Julia $arguments`

#lets run csv2julia (requires python to be installed)
run(cmd)

#pop the outputs in a modelFiles folder
mkpath("modelFiles/averaged/")
mv("odeApoptosis.jl","modelFiles/averaged/odeApoptosis.jl", force=true)
mv("variableNames.jl","modelFiles/averaged/variableNamesApoptosis.jl", force=true);

In [ ]:
function initConditionsApop(y0,syms)
   #units: M 
   y0[findfirst(isequal("DIS"),syms)]=0
   y0[findfirst(isequal("Bid"),syms)]=60000
   y0[findfirst(isequal("Puma"),syms)]=3600 
   y0[findfirst(isequal("Noxa"),syms)]=1800 
   y0[findfirst(isequal("BclxLc"),syms)]=20000
   y0[findfirst(isequal("Bim"),syms)]=2000 
   y0[findfirst(isequal("Mcl1c"),syms)]=5000
   y0[findfirst(isequal("A1c"),syms)]=1600
   y0[findfirst(isequal("Bcl2c"),syms)]=1400
   #this is new and will be replaced by cRel control in multiscale modeling
   y0[findfirst(isequal("Bcl2t"),syms)]=69 
   y0[findfirst(isequal("Bax"),syms)]=14200 
   y0[findfirst(isequal("MBak"),syms)]=4600 
   y0[findfirst(isequal("Mito"),syms)]=500000 
   y0[findfirst(isequal("Inhib_eff_Bcl2"),syms)]=1
   y0[findfirst(isequal("Inhib_eff_BclxL"),syms)]=1
   y0[findfirst(isequal("Inhib_eff_Mcl1"),syms)]=1 

   return y0
end


In [ ]:
include("modelFiles/averaged/odeApoptosis.jl")
include("modelFiles/averaged/variableNamesApoptosis.jl")
p=1
maxTimeSS=10000.0
maxTimeTC=24*60.0
params=1;

In [ ]:
include("utilityFunctions/fixSpecies.jl")
fixSpecies("modelFiles/averaged/odeApoptosis.jl","modelFiles/averaged/odeApoptosisFixed.jl",1)

In [ ]:
include("modelFiles/averaged/odeApoptosisFixed.jl")
stringSyms=syms
y0=zeros(length(syms))
y0=initConditionsApop(y0,syms)
syms2=Symbol.(syms)
f=ODEFunction(odeFile!,syms=syms2)

#steady state phase
print("Have errors with SS code so just using a long TC instead\n")
# prob=ODEProblem(f,y0,(0.0,maxTimeSS))
prob=ODEProblem(f,y0,(0.0,10000.0),abstol=1e-16,reltol=1e-16)
solss=solve(prob,saveat=100.0,progress = true,isoutofdomain=(y,p,t)->any(x->x<0,y))
# solss=solve(prob,saveat=100.0,progress = true)

#save the last bit of the SS solution
dfss = DataFrame(solss[end-100:end],:auto)
#add the variable names and save to a file
insertcols!(dfss, 1, :names=>syms2)
mkpath("outputFiles/apoptosis/")
CSV.write("outputFiles/apoptosis/SS_averaged.csv",dfss);
print("Steady state found\n")

In [ ]:
#dynamic phase, use SS solution as initial conditions
y0=vec(convert(Array, dfss[:,end]))
h(p,t)=y0

In [ ]:
newInit=y0
newInit[1]=y0[1]+0.01

#we might use this later to remove Bcl2 survival signal
#newInit[end]=2.77
prob=ODEProblem(f,newInit,(0.0,maxTimeTC),abstol=1e-16,reltol=1e-16,maxiters=1e9)
sol=solve(prob,saveat=1.0,progress = true)

#lets save this result in a csv file
df=DataFrame(sol)
# insertcols!(df, 1, :names=>syms)
CSV.write("outputFiles/apoptosis/TC_averaged.csv",df)

In [ ]:
#reimport variables names because we want a string again not a sym
include("modelFiles/averaged/variableNamesApoptosis.jl")

speciesToPlot="AMito"
varIndex=findfirst(isequal(speciesToPlot),syms)

In [ ]:
ligandDoses=[0.1,1,10,50,100,200,300,400,500,1000]
# ligandDoses=[0.01,0.1]
# ligandDoses=[0,0.1,1,2,3,4,5,6,7,8,9,10,100]

# y0[1]=0
#store all the results in an array to plot at the end, faster than plotting in the loop.
MOMParray_averaged=zeros(1,Int(maxTimeTC)+1)

for i in ligandDoses
    y0=vec(convert(Array, dfss[:,end]))
    println("Solving solution with ligand: "*string(i)*"")
    newInit=y0
    newInit[1]=y0[1]+i
#     println("Solving solution with ligand: "*string(newInit[1])*"")

    #we might use this later to remove Bcl2 survival signal
    #newInit[end]=2.77
#     println("about to solve")
    prob=ODEProblem(f,newInit,(0.0,maxTimeTC),abstol=1e-16,reltol=1e-16,maxiters=1e9)
    sol=solve(prob,saveat=1,progress = true)
#     println("solved")

    #lets save this result in a csv file
    df=DataFrame(sol)
#     println("Write to csv")
#     insertcols!(df, 1, :names=>syms)
    CSV.write("outputFiles/apoptosis/out_Bcl2_averaged"*string(i)*".csv",df)
#     println("CSV")
    
    thisMOMP_averaged=sol[varIndex,:]
    MOMParray_averaged=[MOMParray_averaged;thisMOMP_averaged']
#     append!(MOMParray, sol(i))

    
end

#cut off the zero array
MOMParray_averaged=MOMParray_averaged[2:end,:];


In [ ]:
# plot and pretify

plot(MOMParray_averaged',palette=palette([:blue, :red], length(ligandDoses)), 
    lw=2,labels=string.(ligandDoses'),legendtitle = "Signal",legend=:outerright,
    size=(900,400),margin=5Plots.mm)
s=string.(collect(0:6:24))
plot!(xticks = (0:(60*6):(24*60),s), ylabel="MOMP (units)",xlabel="time h", 
    xtickfontsize=14,ytickfontsize=14,xguidefontsize=18,yguidefontsize=18,legendfontsize=14,dpi=300,fmt = :png)
# png("outputFiles/apoptosis/averaged_MOMP")

## Bcl2 inhibition

In [ ]:
include("modelFiles/averaged/odeApoptosisFixed.jl")
stringSyms=syms
y0=zeros(length(syms))
y0=initConditionsApop(y0,syms)
syms2=Symbol.(syms)
f=ODEFunction(odeFile!,syms=syms2)

#steady state phase
print("Have errors with SS code so just using a long TC instead\n")
# prob=ODEProblem(f,y0,(0.0,maxTimeSS))
prob=ODEProblem(f,y0,(0.0,10000.0),abstol=1e-16,reltol=1e-16)
solss=solve(prob,saveat=100.0,progress = true,isoutofdomain=(y,p,t)->any(x->x<0,y))
# solss=solve(prob,saveat=100.0,progress = true)

#save the last bit of the SS solution
dfss = DataFrame(solss[end-100:end],:auto)
#add the variable names and save to a file
insertcols!(dfss, 1, :names=>syms2)
mkpath("outputFiles/apoptosis/")
CSV.write("outputFiles/apoptosis/SS_averaged.csv",dfss);
print("Steady state found\n")

In [ ]:
#dynamic phase, use SS solution as initial conditions
y0=vec(convert(Array, dfss[:,end]))
h(p,t)=y0

In [ ]:
y0[findfirst(isequal("Inhib_eff_Bcl2"),syms)]=0.5

In [ ]:
newInit=y0
newInit[1]=y0[1]+0.01

#we might use this later to remove Bcl2 survival signal
#newInit[end]=2.77
prob=ODEProblem(f,newInit,(0.0,maxTimeTC),abstol=1e-16,reltol=1e-16,maxiters=1e9)
sol_Bcl2_05=solve(prob,saveat=1.0,progress = true)

#lets save this result in a csv file
df=DataFrame(sol_Bcl2_05)
# insertcols!(df, 1, :names=>syms)
CSV.write("outputFiles/apoptosis/TC_average_Bcl2_05.csv",df)

In [ ]:
include("modelFiles/averaged/odeApoptosisFixed.jl")
stringSyms=syms
y0=zeros(length(syms))
y0=initConditionsApop(y0,syms)
syms2=Symbol.(syms)
f=ODEFunction(odeFile!,syms=syms2)

#steady state phase
print("Have errors with SS code so just using a long TC instead\n")
# prob=ODEProblem(f,y0,(0.0,maxTimeSS))
prob=ODEProblem(f,y0,(0.0,10000.0),abstol=1e-16,reltol=1e-16)
solss=solve(prob,saveat=100.0,progress = true,isoutofdomain=(y,p,t)->any(x->x<0,y))
# solss=solve(prob,saveat=100.0,progress = true)

#save the last bit of the SS solution
dfss = DataFrame(solss[end-100:end],:auto)
#add the variable names and save to a file
insertcols!(dfss, 1, :names=>syms2)
mkpath("outputFiles/apoptosis/")
CSV.write("outputFiles/apoptosis/SS_averaged.csv",dfss);
print("Steady state found\n")

In [ ]:
#dynamic phase, use SS solution as initial conditions
y0=vec(convert(Array, dfss[:,end]))
h(p,t)=y0

In [ ]:
y0[findfirst(isequal("Inhib_eff_Bcl2"),syms)]=0.8

In [ ]:
newInit=y0
newInit[1]=y0[1]+0.01

#we might use this later to remove Bcl2 survival signal
#newInit[end]=2.77
prob=ODEProblem(f,newInit,(0.0,maxTimeTC),abstol=1e-16,reltol=1e-16,maxiters=1e9)
sol_Bcl2_08=solve(prob,saveat=1.0,progress = true)

#lets save this result in a csv file
df=DataFrame(sol_Bcl2_08)
# insertcols!(df, 1, :names=>syms)
CSV.write("outputFiles/apoptosis/TC_average_Bcl2_08.csv",df)

## BclxL inhibition

In [ ]:
include("modelFiles/averaged/odeApoptosisFixed.jl")
stringSyms=syms
y0=zeros(length(syms))
y0=initConditionsApop(y0,syms)
syms2=Symbol.(syms)
f=ODEFunction(odeFile!,syms=syms2)

#steady state phase
print("Have errors with SS code so just using a long TC instead\n")
# prob=ODEProblem(f,y0,(0.0,maxTimeSS))
prob=ODEProblem(f,y0,(0.0,10000.0),abstol=1e-16,reltol=1e-16)
solss=solve(prob,saveat=100.0,progress = true,isoutofdomain=(y,p,t)->any(x->x<0,y))
# solss=solve(prob,saveat=100.0,progress = true)

#save the last bit of the SS solution
dfss = DataFrame(solss[end-100:end],:auto)
#add the variable names and save to a file
insertcols!(dfss, 1, :names=>syms2)
mkpath("outputFiles/apoptosis/")
CSV.write("outputFiles/apoptosis/SS_averaged.csv",dfss);
print("Steady state found\n")

In [ ]:
#dynamic phase, use SS solution as initial conditions
y0=vec(convert(Array, dfss[:,end]))
h(p,t)=y0

In [ ]:
y0[findfirst(isequal("Inhib_eff_BclxL"),syms)]=0.5

In [ ]:
newInit=y0
newInit[1]=y0[1]+0.01

#we might use this later to remove Bcl2 survival signal
#newInit[end]=2.77
prob=ODEProblem(f,newInit,(0.0,maxTimeTC),abstol=1e-16,reltol=1e-16,maxiters=1e9)
sol_BclxL_05=solve(prob,saveat=1.0,progress = true)

#lets save this result in a csv file
df=DataFrame(sol_BclxL_05)
# insertcols!(df, 1, :names=>syms)
CSV.write("outputFiles/apoptosis/TC_average_BclxL_05.csv",df)

In [ ]:
include("modelFiles/averaged/odeApoptosisFixed.jl")
stringSyms=syms
y0=zeros(length(syms))
y0=initConditionsApop(y0,syms)
syms2=Symbol.(syms)
f=ODEFunction(odeFile!,syms=syms2)

#steady state phase
print("Have errors with SS code so just using a long TC instead\n")
# prob=ODEProblem(f,y0,(0.0,maxTimeSS))
prob=ODEProblem(f,y0,(0.0,10000.0),abstol=1e-16,reltol=1e-16)
solss=solve(prob,saveat=100.0,progress = true,isoutofdomain=(y,p,t)->any(x->x<0,y))
# solss=solve(prob,saveat=100.0,progress = true)

#save the last bit of the SS solution
dfss = DataFrame(solss[end-100:end],:auto)
#add the variable names and save to a file
insertcols!(dfss, 1, :names=>syms2)
mkpath("outputFiles/apoptosis/")
CSV.write("outputFiles/apoptosis/SS_averaged.csv",dfss);
print("Steady state found\n")

In [ ]:
#dynamic phase, use SS solution as initial conditions
y0=vec(convert(Array, dfss[:,end]))
h(p,t)=y0

In [ ]:
y0[findfirst(isequal("Inhib_eff_BclxL"),syms)]=0.8

In [ ]:
newInit=y0
newInit[1]=y0[1]+0.01

#we might use this later to remove Bcl2 survival signal
#newInit[end]=2.77
prob=ODEProblem(f,newInit,(0.0,maxTimeTC),abstol=1e-16,reltol=1e-16,maxiters=1e9)
sol_BclxL_08=solve(prob,saveat=1.0,progress = true)

#lets save this result in a csv file
df=DataFrame(sol_BclxL_08)
# insertcols!(df, 1, :names=>syms)
CSV.write("outputFiles/apoptosis/TC_average_BclxL_08.csv",df)

## Mcl1 inhibition

In [ ]:
include("modelFiles/averaged/odeApoptosisFixed.jl")
stringSyms=syms
y0=zeros(length(syms))
y0=initConditionsApop(y0,syms)
syms2=Symbol.(syms)
f=ODEFunction(odeFile!,syms=syms2)

#steady state phase
print("Have errors with SS code so just using a long TC instead\n")
# prob=ODEProblem(f,y0,(0.0,maxTimeSS))
prob=ODEProblem(f,y0,(0.0,10000.0),abstol=1e-16,reltol=1e-16)
solss=solve(prob,saveat=100.0,progress = true,isoutofdomain=(y,p,t)->any(x->x<0,y))
# solss=solve(prob,saveat=100.0,progress = true)

#save the last bit of the SS solution
dfss = DataFrame(solss[end-100:end],:auto)
#add the variable names and save to a file
insertcols!(dfss, 1, :names=>syms2)
mkpath("outputFiles/apoptosis/")
CSV.write("outputFiles/apoptosis/SS_averaged.csv",dfss);
print("Steady state found\n")

In [ ]:
#dynamic phase, use SS solution as initial conditions
y0=vec(convert(Array, dfss[:,end]))
h(p,t)=y0

In [ ]:
y0[findfirst(isequal("Inhib_eff_Mcl1"),syms)]=0.5

In [ ]:
newInit=y0
newInit[1]=y0[1]+0.01

#we might use this later to remove Bcl2 survival signal
#newInit[end]=2.77
prob=ODEProblem(f,newInit,(0.0,maxTimeTC),abstol=1e-16,reltol=1e-16,maxiters=1e9)
sol_Mcl1_05=solve(prob,saveat=1.0,progress = true)

#lets save this result in a csv file
df=DataFrame(sol_Mcl1_05)
# insertcols!(df, 1, :names=>syms)
CSV.write("outputFiles/apoptosis/TC_average_Mcl1_05.csv",df)

In [ ]:
include("modelFiles/averaged/odeApoptosisFixed.jl")
stringSyms=syms
y0=zeros(length(syms))
y0=initConditionsApop(y0,syms)
syms2=Symbol.(syms)
f=ODEFunction(odeFile!,syms=syms2)

#steady state phase
print("Have errors with SS code so just using a long TC instead\n")
# prob=ODEProblem(f,y0,(0.0,maxTimeSS))
prob=ODEProblem(f,y0,(0.0,10000.0),abstol=1e-16,reltol=1e-16)
solss=solve(prob,saveat=100.0,progress = true,isoutofdomain=(y,p,t)->any(x->x<0,y))
# solss=solve(prob,saveat=100.0,progress = true)

#save the last bit of the SS solution
dfss = DataFrame(solss[end-100:end],:auto)
#add the variable names and save to a file
insertcols!(dfss, 1, :names=>syms2)
mkpath("outputFiles/apoptosis/")
CSV.write("outputFiles/apoptosis/SS_averaged.csv",dfss);
print("Steady state found\n")

In [ ]:
#dynamic phase, use SS solution as initial conditions
y0=vec(convert(Array, dfss[:,end]))
h(p,t)=y0

In [ ]:
y0[findfirst(isequal("Inhib_eff_Mcl1"),syms)]=0.8

In [ ]:
newInit=y0
newInit[1]=y0[1]+0.01

#we might use this later to remove Bcl2 survival signal
#newInit[end]=2.77
prob=ODEProblem(f,newInit,(0.0,maxTimeTC),abstol=1e-16,reltol=1e-16,maxiters=1e9)
sol_Mcl1_08=solve(prob,saveat=1.0,progress = true)

#lets save this result in a csv file
df=DataFrame(sol_Mcl1_08)
# insertcols!(df, 1, :names=>syms)
CSV.write("outputFiles/apoptosis/TC_average_Mcl1_08.csv",df)

In [ ]:
AMito=findfirst(isequal("AMito"),syms)
plot(sol[AMito,:]./sol[AMito,:],yaxis=:log,lw=2,linecolor="Black",label="WT")
plot!(sol_Bcl2_05[AMito,:]./sol[AMito,:],yaxis=:log,lw=2,linecolor="Olive Drab",label="Bcl-2 (50%)")
plot!(sol_Bcl2_08[AMito,:]./sol[AMito,:],yaxis=:log,lw=2,linecolor="Tomato",label="Bcl-2 (20%)")
plot!(sol_BclxL_05[AMito,:]./sol[AMito,:],yaxis=:log,lw=2,linecolor="Light Sky Blue",label="Bcl-xL (50%)")
plot!(sol_BclxL_08[AMito,:]./sol[AMito,:],yaxis=:log,lw=2,linecolor="Orchid",label="Bcl-xL (20%)")
plot!(sol_Mcl1_05[AMito,:]./sol[AMito,:],yaxis=:log,lw=2,linecolor="Slate Gray",label="Mcl1 (50%)")
plot!(sol_Mcl1_08[AMito,:]./sol[AMito,:],yaxis=:log,lw=2,linecolor="Orange",label="Mcl1 (20%)")
plot!(palette=:seaborn_colorblind,legendtitle = "Inhibitors",legend=:outerright,size=(900,400),margin=5Plots.mm)
s=string.(collect(0:8:24))
plot!(xticks = (0:(60*8):(24*60),s), ylabel="MOMP (Log Fold change)",xlabel="time h", 
    xtickfontsize=14,ytickfontsize=14,xguidefontsize=18,yguidefontsize=18,legendfontsize=14,dpi=300,fmt = :png)
# png("outputFiles/apoptosis/Inhib_average")